<a href="https://colab.research.google.com/github/anushahulbatte/deeplearning/blob/main/DL_taska.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hello world')

hello world
